<a href="https://colab.research.google.com/github/100495802/G11.AA-495802-495702/blob/main/P1AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este notebook contiene toda la parte de análisis y examinación de los datos, y se centra en la comprensión y resumen de éstos. En este notebook encontraremos todo el proceso de creacion del EDA, ajuste de hiperparámetros, selección del modelo, y resolución de las preguntas establecidas en el enunciado de la práctica.

# 1. Github y Carga de Datos

Primeramente, nos disponemos a realizar el [repositorio de Github](https://github.com/100495802/G11.AA-495802-495702.git) correspondiente al proyecto, el cual se puede consultar clickando en el enlace.

Seguidamente, cargamos los datos elegidos según el estándar especificado para la práctica (suma de los dos últimos dígitos del NIA de un alumno: _100495702 -> 0+2=2_).

## 1.1. Carga de datos

In [2]:
import pandas as pd
datos = pd.read_csv("attrition_availabledata_02.csv.gz", compression="gzip", sep=",")

# 2. EDA Simplificado

En esta sección vamos a comentar la descripción de los distintos datos que hay en nuestra base de datos. Diferenciaremos que variable son categóricas, ordinales o numéricas; cuáles tienen alta cardinalidad, cuáles tienen valores faltantes; si hay columnas de ID; y si es un problema de regresión o de evaluación.

Mediante el atributo info() vamos a obtener los tipos de datos que hay en nuestra base de datos y asi poder evaluar los datos.

In [3]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2940 entries, 0 to 2939
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   hrs                      2940 non-null   float64
 1   absences                 2940 non-null   int64  
 2   JobInvolvement           2940 non-null   int64  
 3   PerformanceRating        2940 non-null   int64  
 4   EnvironmentSatisfaction  2927 non-null   float64
 5   JobSatisfaction          2925 non-null   float64
 6   WorkLifeBalance          2911 non-null   float64
 7   Age                      2940 non-null   int64  
 8   BusinessTravel           2940 non-null   object 
 9   Department               2940 non-null   object 
 10  DistanceFromHome         2940 non-null   int64  
 11  Education                2940 non-null   int64  
 12  EducationField           2940 non-null   object 
 13  EmployeeCount            2940 non-null   int64  
 14  EmployeeID              

In [4]:
datos.describe()

,hrs,absences,JobInvolvement,PerformanceRating,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,Age,DistanceFromHome,Education,...,MonthlyIncome,NumCompaniesWorked,PercentSalaryHike,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager
count,2940.000000,2940.000000,2940.000000,2940.000000,2927.000000,2925.000000,2911.000000,2940.000000,2940.000000,2940.000000,...,2940.000000,2926.000000,2940.000000,2940.0,2940.000000,2935.000000,2940.000000,2940.000000,2940.000000,2940.000000
mean,7.322768,12.706803,2.728571,3.154422,2.723266,2.746325,2.760907,36.861224,9.305102,2.906463,...,64379.826531,2.664388,15.187075,8.0,0.785034,11.363884,2.798639,7.095578,2.227891,4.191156
std,1.335600,5.533199,0.716167,0.361414,1.096170,1.104612,0.713539,9.286733,8.201638,1.023254,...,47021.197990,2.495153,3.661275,0.0,0.851343,7.897032,1.304166,6.161878,3.274101,3.627734
min,5.416880,1.000000,1.000000,3.000000,1.000000,1.000000,1.000000,18.000000,1.000000,1.000000,...,10090.000000,0.000000,11.000000,8.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6.272786,8.000000,2.000000,3.000000,2.000000,2.000000,2.000000,30.000000,2.000000,2.000000,...,28750.000000,1.000000,12.000000,8.0,0.000000,6.000000,2.000000,3.000000,0.000000,2.000000
50%,7.032627,13.000000,3.000000,3.000000,3.000000,3.000000,3.000000,35.000000,7.000000,3.000000,...,48340.000000,2.000000,14.000000,8.0,1.000000,10.000000,3.000000,5.000000,1.000000,3.000000
75%,7.948416,17.000000,3.000000,3.000000,4.000000,4.000000,3.000000,43.000000,14.000000,4.000000,...,80080.000000,4.000000,18.000000,8.0,1.000000,16.000000,3.000000,10.000000,3.000000,7.000000
max,10.937261,24.000000,4.000000,4.000000,4.000000,4.000000,4.000000,60.000000,29.000000,5.000000,...,199990.000000,9.000000,25.000000,8.0,3.000000,40.000000,6.000000,40.000000,15.000000,17.000000


## 2.1. Variables numéricas y categóricas

Con los datos visualizados en el info, vamos a observar los valores de las variables más a fondo, haciendo hincapién en las distintas observaciones que se piden en la práctica, empezamos visualizando los valores de las variables númericas

In [5]:
numéricas = datos.select_dtypes(include=['int64', 'float64']).columns
print(numéricas.tolist())

['hrs', 'absences', 'JobInvolvement', 'PerformanceRating', 'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance', 'Age', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeID', 'JobLevel', 'MonthlyIncome', 'NumCompaniesWorked', 'PercentSalaryHike', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'YearsAtCompany', 'YearsSinceLastPromotion', 'YearsWithCurrManager']


Ahora hacemos lo mismo pero con las variables cadenas de texto

In [6]:
categóricas = datos.select_dtypes(include=["object"]).columns.drop("Attrition")
print(categóricas.tolist())

['BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'Over18']


## 2.2. Variables con alta cardinalidad

In [7]:
for columna in datos.select_dtypes(include=["object"]).columns:
    print(columna)
    print(datos[columna].unique())

BusinessTravel
['Travel_Frequently' 'Non-Travel' 'Travel_Rarely']
Department
['Research & Development' 'Sales' 'Human Resources']
EducationField
['Life Sciences' 'Medical' 'Other' 'Technical Degree' 'Marketing'
 'Human Resources']
Gender
['Male' 'Female']
JobRole
['Laboratory Technician' 'Healthcare Representative' 'Research Scientist'
 'Sales Representative' 'Manufacturing Director' 'Sales Executive'
 'Research Director' 'Human Resources' 'Manager']
MaritalStatus
['Married' 'Divorced' 'Single']
Over18
['Y']
Attrition
['Yes' 'No']


Analizando el texto impreso podemos observar que variables categóricas presentan elevada cardinalidad. Consideraremos elevado un número de opciones mayor de 4, en este caso salen: EducationField y JobRole.

## 2.3. Variables nulas

A continuación podemos comprobar si existen valores nulos en las variables y cuantos hay en cada una de ellas.

In [8]:
nulos = datos.isnull().sum()
nulos[nulos > 0]

,0
EnvironmentSatisfaction,13
JobSatisfaction,15
WorkLifeBalance,29
NumCompaniesWorked,14
TotalWorkingYears,5


Como observamos tenemos 5 variables con valores nulos en sus filas. Teniendo en cuenta que en total hay 2900 filas en la tabla se pueden hacer fácilmente los cálculos de cuántas filas vacias hay por variable.

Una vez analizado todo lo anterior podemos buscar que posibles columnas actúan como id en la base de datos. Observando los nombres de las variables una posible opción sería EmployeeID aunque queremos comprobar si actúa como ID alguna otra.

## 2.4. Variables de identificación ID

In [9]:
for columna in datos.columns:
    if datos[columna].nunique() == len(datos):
        print(f"La columna '{columna}' es un posible identificador único.")

La columna 'EmployeeID' es un posible identificador único.


Por último, el problema que se nos propone es de clasificación, pues contamos con la variable Attrition que determina si un empleado serñia propenso a abandonar la empresa en función del resto de datos asociados. Vamos a comprobar si se trata de un problema desbalanceado o no.

## 2.5. ¿Problema desbalanceado?

In [10]:
datos["Attrition"].value_counts()

,count
Attrition,
No,2466
Yes,474


Como observamos, el conteo de datos nos demuestra que es un problema desbalanceado hacia el no, siendo más del 80% de las clasificaciones de un tipo.

# 3. Decidir como se va a realizar la evaluación

Con los datos ya analizados, vamos a continuar con la realización de la práctica y vamos a comentar como vamos a evaluar nuestro modelo.

## 3.1. Planificación de la evaluación

- **División de los datos:**

  Para realizar la evaluación de nuestro modelo lo primero que tenemos que hacer es dividir el conjunto de datos en dos partes: train y test. Usaremos aproximadamente 2/3 de los datos para train y el tercio restante para los test.

- **Preprocesado de datos:**

  Con los datos del train crearemos el modelo con el que evaluar los test. Para ello primero usaremos KNN con los default HP. Modificaremos el escalado usando varios formatos como Standard, MinMax y Robust. También modificaremos el tratado de datos vacios (la imputación) usando la Media, la Mediana... Por último en todos los casos usaremos el modelo 3-Fold con crossvalidation. Con todo esto buscamos obtener el mejor Escaler-Imputer.

- **Ajuste de hyperparametro HPO:**

  Una vez hecho esto comenzamos la búsqueda del mejor modelo; construimos una serie de modelos basandonos en las metodologías de KNN y árboles de  decisión incluyendo HPO (ajuste de hiperparametros) o no. Una vez creados todos elegiremos el modelo que mejor se ajuste a los datos procurando evitar cualquier data leakage y sobreajuste.

- **Evaluación del modelo:**

  Cuando hayamos encontrado la mejor alternativa usaremos el conjunto de test para obtener una estimación y rendimiento futuro del modelo. Esto es lo que se conoce como la evaluación del modelo


## 3.2. Planificación de las predicciones

Todo esto en conjunto lleva el modelo final, con el cual realizaremos predicciones usando un conjunto de datos de competición (NoteBook 2)

# Métodos Básicos: KNN y TREES

In [41]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from  sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, RobustScaler, StandardScaler
import time

In [35]:
X = datos.drop("Attrition", axis=1)
y = datos["Attrition"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)

## 4.1. Búsqueda de mejor método de escalado e imputación

Comenzamos usando KNN con los distintos métodos de escalado y de imputación con el objetivo de obtener los más apropiados para el problema.

In [36]:
Scalers = [StandardScaler(), MinMaxScaler(), RobustScaler()]
Imputers = [SimpleImputer(strategy="mean"), SimpleImputer(strategy="median")]
Mejor_Scaler = None
Mejor_Imputer = None
Mejor_Score = 0
categorical_transformer = Pipeline(steps=[("encoder", OneHotEncoder(handle_unknown="ignore"))])
for scaler in Scalers:
    for imputer in Imputers:
      numerical_transformer = Pipeline(steps=[("imputer", imputer), ("scaler", scaler)])
      preprocessor = ColumnTransformer(transformers=[("num", numerical_transformer, numéricas), ("cat", categorical_transformer, categóricas)])
      pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", KNeighborsClassifier())])
      score = cross_val_score(pipeline, X_train, y_train, cv=3, scoring="balanced_accuracy").mean()
      if score.mean() > Mejor_Score:
        Mejor_Score = score.mean()
        Mejor_Scaler = scaler
        Mejor_Imputer = imputer
      print(f"Scaler: {scaler}, Imputer: {imputer}, Score: {score}")
print(f"Mejor Scaler: {Mejor_Scaler}, Mejor Imputer: {Mejor_Imputer}, Mejor Score: {Mejor_Score}")

Scaler: StandardScaler(), Imputer: SimpleImputer(), Score: 0.5761237407807159
Scaler: StandardScaler(), Imputer: SimpleImputer(strategy='median'), Score: 0.5764301133297356
Scaler: MinMaxScaler(), Imputer: SimpleImputer(), Score: 0.573666014570966
Scaler: MinMaxScaler(), Imputer: SimpleImputer(strategy='median'), Score: 0.573666014570966
Scaler: RobustScaler(), Imputer: SimpleImputer(), Score: 0.5883600917431192
Scaler: RobustScaler(), Imputer: SimpleImputer(strategy='median'), Score: 0.5834665632307968
Mejor Scaler: RobustScaler(), Mejor Imputer: SimpleImputer(), Mejor Score: 0.5883600917431192


Observando los datos podemos decir que el mejor método de escalado es Robust, mientras que el método de imputación es la media.

## 4.2. Busqueda de hiperparámetros

Una vez tenemos los métodos de escalado y de imputación seleccionados podemos pasar a la siguiente fase. Ahora buscaremos los hiperparametros que mejor se ajustan al problema

In [42]:
clasificadores = [KNeighborsClassifier(), DecisionTreeClassifier()]
categorical_transformer = Pipeline(steps=[("encoder", OneHotEncoder(handle_unknown="ignore"))])
numerical_transformer = Pipeline(steps=[("imputer", Mejor_Imputer), ("scaler", Mejor_Scaler)])
preprocessor = ColumnTransformer(transformers=[("num", numerical_transformer, numéricas), ("cat", categorical_transformer, categóricas)])
for clasificador in clasificadores:
    pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", clasificador)])
    if isinstance(clasificador, KNeighborsClassifier):
            param_grid = {
                'classifier__n_neighbors': [3, 5, 7, 9, 11],
                'classifier__weights': ['uniform', 'distance'],
                'classifier__metric': ['minkowski', 'euclidean', 'manhattan']
            }
    elif isinstance(clasificador, DecisionTreeClassifier):
            param_grid = {
                'classifier__criterion': ['gini', 'entropy'],
                'classifier__max_depth': [5, 10, 15, None],
                'classifier__min_samples_split': [2, 5, 10],
                'classifier__min_impurity_decrease': [0.0, 0.01, 0.02]
            }
    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_test = label_encoder.transform(y_test)
    grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, scoring='balanced_accuracy')
    start_time = time.time()
    grid_search.fit(X_train, y_train)
    end_time = time.time()

    # Mostrar los mejores hiperparámetros
    print("Mejores parámetros:", grid_search.best_params_)
    print("Mejor puntuación:", grid_search.best_score_)
    print("Tiempo de ejecución:", end_time - start_time, "segundos")

Mejores parámetros: {'classifier__metric': 'manhattan', 'classifier__n_neighbors': 5, 'classifier__weights': 'distance'}
Mejor puntuación: 0.8067087155963303
Tiempo de ejecución: 10.171767711639404 segundos
Mejores parámetros: {'classifier__criterion': 'entropy', 'classifier__max_depth': 15, 'classifier__min_impurity_decrease': 0.0, 'classifier__min_samples_split': 2}
Mejor puntuación: 0.7678303202014751
Tiempo de ejecución: 14.579367637634277 segundos


Vamos a comprobar los resultados obtenidos con el formato dummy

In [53]:
y_train = datos["Attrition"].iloc[X_train.index]  # Assuming X_train.index holds the original indices
y_test = datos["Attrition"].iloc[X_test.index]  # Assuming X_test.index holds the original indices

pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", DummyClassifier())])
param_grid = {
    'classifier__strategy': ['most_frequent', 'stratified', 'prior', 'uniform',  # Removed 'constant'
                            'constant'],  # Added 'constant' back with a constant value
    'classifier__constant': [0, 1] if 'constant' in ['most_frequent', 'stratified', 'prior', 'uniform', 'constant'] else [] # Specify constant values only when 'constant' strategy is used
}
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)  # Encode y_train for the DummyClassifier
y_test_encoded = label_encoder.transform(y_test)
grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, scoring='balanced_accuracy', error_score='raise')
start_time = time.time()
grid_search.fit(X_train, y_train_encoded)
dummy_pred = grid_search.predict(X_test)
end_time = time.time()

print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntuación:", grid_search.best_score_)
print("Tiempo de ejecución:", end_time - start_time, "segundos")

Mejores parámetros: {'classifier__constant': 0, 'classifier__strategy': 'stratified'}
Mejor puntuación: 0.521917723511423
Tiempo de ejecución: 8.059552431106567 segundos


# Avanzados: Modelos lineales y SVM's